In [1]:
import os, sys
import pandas as pd

#import the functions
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import rtcw_functions as RTCW

db_name = 'rtcw_db.db'
demos_folder = 'demos'

cwd = os.getcwd()
demos_path = os.path.join(cwd, '../', demos_folder)

exe_name = 'Anders.Gaming.LibTech3.exe'
demos_folder = 'demos'
root_path = 'C:\\Users\\JelleGrammens\\Documents\\GIT\\rtcw_demo_analyzer' #for executable we need a direct path

In [2]:
#get demos_dct
demos_dct = RTCW.make_dictionary(demos_path)

parsed 100 matches
parsed 200 matches
parsed 300 matches
parsed 400 matches
finished parsing all matches!


In [3]:
#load obituaries in a df
import sqlite3_connector
reload(sqlite3_connector)
db = sqlite3_connector.sqlite3_connector(os.path.join(root_path, db_name))

In [23]:
obituary_df = db.get_table_as_df(table_name= 'obituary')
player_df = db.get_table_as_df(table_name= 'player')
obituary_df = RTCW.add_match_data(obituary_df, player_df, demos_dct)
obituary_df = obituary_df.loc[obituary_df['bAttacker'] == 0]
#obituary_df = obituary_df.loc[obituary_df['League'] == 'qcon02']


obituary table fetched in 0.57 seconds
player table fetched in 0.05 seconds


In [26]:
obituary_df.loc[obituary_df['bAttacker'] == 0]

,dwSeq,szMd5,bAttacker,bTarget,bWeapon,dwTime,bIsTeamkill,szTimeString,szMessage,Date,Importance,League,Shoutcast,TeamA,TeamB,demoName,matchName,szCleanName
782,783,59b1f2608f22e9e90e944b36f5929c7c,0,10,16,55279950,0,7:36,|TFP|AbDuL was killed by CoH]I[Smoove's .45ACP...,2002-08-13,None,liga-sw-s1,False,the-fearless-pigs,circle-of-haste,cohvstfp.dm_59,rtcw_2002.08.13_liga-sw-s1_the-fearless-pigs_v...,CoH]I[Smoove
799,800,59b1f2608f22e9e90e944b36f5929c7c,0,10,18,55339050,0,6:36,|TFP|AbDuL was killed by CoH]I[Smoove's Thompson,2002-08-13,None,liga-sw-s1,False,the-fearless-pigs,circle-of-haste,cohvstfp.dm_59,rtcw_2002.08.13_liga-sw-s1_the-fearless-pigs_v...,CoH]I[Smoove
823,824,59b1f2608f22e9e90e944b36f5929c7c,0,8,18,55444500,0,4:51,|TFP|RED BARON was killed by CoH]I[Smoove's Th...,2002-08-13,None,liga-sw-s1,False,the-fearless-pigs,circle-of-haste,cohvstfp.dm_59,rtcw_2002.08.13_liga-sw-s1_the-fearless-pigs_v...,CoH]I[Smoove
824,825,59b1f2608f22e9e90e944b36f5929c7c,0,10,18,55450600,0,4:45,|TFP|AbDuL was killed by CoH]I[Smoove's Thompson,2002-08-13,None,liga-sw-s1,False,the-fearless-pigs,circle-of-haste,cohvstfp.dm_59,rtcw_2002.08.13_liga-sw-s1_the-fearless-pigs_v...,CoH]I[Smoove
849,850,59b1f2608f22e9e90e944b36f5929c7c,0,11,18,55535850,0,3:20,|TFP|REDHEAT was killed by CoH]I[Smoove's Thom...,2002-08-13,None,liga-sw-s1,False,the-fearless-pigs,circle-of-haste,cohvstfp.dm_59,rtcw_2002.08.13_liga-sw-s1_the-fearless-pigs_v...,CoH]I[Smoove
851,852,59b1f2608f22e9e90e944b36f5929c7c,0,8,16,55546150,0,3:09,|TFP|RED BARON was killed by CoH]I[Smoove's .4...,2002-08-13,None,liga-sw-s1,False,the-fearless-pigs,circle-of-haste,cohvstfp.dm_59,rtcw_2002.08.13_liga-sw-s1_the-fearless-pigs_v...,CoH]I[Smoove
860,861,59b1f2608f22e9e90e944b36f5929c7c,0,11,18,55577150,0,2:38,|TFP|REDHEAT was killed by CoH]I[Smoove's Thom...,2002-08-13,None,liga-sw-s1,False,the-fearless-pigs,circle-of-haste,cohvstfp.dm_59,rtcw_2002.08.13_liga-sw-s1_the-fearless-pigs_v...,CoH]I[Smoove
864,865,59b1f2608f22e9e90e944b36f5929c7c,0,5,5,55588000,0,2:27,|TFP|Jumpmann was exploded by CoH]I[Smoove's g...,2002-08-13,None,liga-sw-s1,False,the-fearless-pigs,circle-of-haste,cohvstfp.dm_59,rtcw_2002.08.13_liga-sw-s1_the-fearless-pigs_v...,CoH]I[Smoove
865,866,59b1f2608f22e9e90e944b36f5929c7c,0,6,5,55588000,0,2:27,|TFP|JayKay was exploded by CoH]I[Smoove's gre...,2002-08-13,None,liga-sw-s1,False,the-fearless-pigs,circle-of-haste,cohvstfp.dm_59,rtcw_2002.08.13_liga-sw-s1_the-fearless-pigs_v...,CoH]I[Smoove
868,869,59b1f2608f22e9e90e944b36f5929c7c,0,10,18,55590900,0,2:25,|TFP|AbDuL was killed by CoH]I[Smoove's Thompson,2002-08-13,None,liga-sw-s1,False,the-fearless-pigs,circle-of-haste,cohvstfp.dm_59,rtcw_2002.08.13_liga-sw-s1_the-fearless-pigs_v...,CoH]I[Smoove


In [40]:
df_spree = RTCW.get_kill_sprees(obituary_df, 
                                maxtime_secs = 10, 
                                include_weapon_filter = ['selfkill', 'grenade'],
                                exclude_weapon_filter = None, 
                                minspree = 3,
                                verbose = True)

scanned 100 demos of 469 demos in total
scanned 200 demos of 469 demos in total
scanned 300 demos of 469 demos in total
scanned 400 demos of 469 demos in total
all done!


In [41]:
df_spree[['match', 'demo', 'player', 'spreecount', 'weapons']]

,match,demo,player,spreecount,weapons
0,rtcw_2002.08.13_liga-sw-s1_the-fearless-pigs_v...,cohvstfp.dm_59,Coh]i[smoove,3,4-5-5-
1,rtcw_2003.09.18_rtcw.no-premier-s6_ff_vs_4kings,222442-WTV-rtcwno-mp_village.dm_60,Ff-marek.zicemat!,3,5-5-5-
2,rtcw_2003.09.23_rtcw.no-premier-s6_4kings_vs_i...,222933-WTV-rtcwno-mp_village.dm_60,4k^ogdoad_0_o,3,5-5-5-
3,rtcw_2002.08.15_qcon02_ocrana_vs_diy_wb-round1,02_mp_base-OCRANAcsDiY_Qcon2.dm_59,Ocrdeep_D-link,3,5-5-5-


In [37]:
RTCW.cut_demos(root_path, demos_dct, df_spree, offset_start = 8, offset_end = 8, transform_to_dm_60 = True)
RTCW.generate_capture_list(df_spree, transform_to_dm_60 = True)

In [ ]:
for demo in df_spree.md5:
    for k in demos_dct:
        if demo in demos_dct[k][0]:
            print k

In [ ]:
reload(RTCW)

In [6]:
obituary_df

,dwSeq,szMd5,bAttacker,bTarget,bWeapon,dwTime,bIsTeamkill,szTimeString,szMessage,Date,Importance,League,Shoutcast,TeamA,TeamB,demoName,matchName,szCleanName


In [24]:

for k in demos_dct:
    if 'mp_sub-222842-[CUP]Veracious.dm_59' in demos_dct[k][1]:
        print demos_dct[k]

[['d0f944f9576532b1209cf709be60ceab', 'e9aa011b54a64f71acad775ec042637a', '7820946197844e1aa9da1affd810b68b', '006786ef8fcbfe3b4003814af4030cfd', 'c2bf0f4d0c10ae1b0f2c86cbe6fa6e14', '4b242661598df625bca615529e0dff1e'], ['mp_sub-221359-[CUP]Veracious.dm_59', 'mp_sub-221811-[CUP]Veracious.dm_59', 'mp_sub-222842-[CUP]Veracious.dm_59', 'mp_assault-223712-[CUP]Veracious.dm_59', 'mp_assault-225623-[CUP]Veracious.dm_59', 'mp_assault-230317-[CUP]Veracious.dm_59']]
